# Source:
 https://github.com/jeffheaton/t81_558_deep_learning/blob/cac591e478a6f19f3d4193d8514bfce89564ae6e/t81_558_class_07_2_train_gan.ipynb

## Mount Google Colab

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 opensimplex

## Fix Issue with StyleGAN2

In [3]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

fatal: destination path 'stylegan2-ada-pytorch' already exists and is not an empty directory.


In [4]:
%cd stylegan2-ada-pytorch/torch_utils/ops/
!rm conv2d_gradfix.py
!rm grid_sample_gradfix.py

/content/stylegan2-ada-pytorch/torch_utils/ops


In [5]:
!cp /content/drive/MyDrive/conv2d_gradfix.py conv2d_gradfix.py
!cp /content/drive/MyDrive/grid_sample_gradfix.py grid_sample_gradfix.py

In [6]:
!ls

bias_act.cpp  bias_act.py	  fma.py		  __pycache__	 upfirdn2d.h
bias_act.cu   conv2d_gradfix.py   grid_sample_gradfix.py  upfirdn2d.cpp  upfirdn2d.py
bias_act.h    conv2d_resample.py  __init__.py		  upfirdn2d.cu


In [7]:
!ls /content/drive/MyDrive/skin_data/images

acne  images_fst_5_and_6  images_fst6


In [8]:
CMD = "python /content/stylegan2-ada-pytorch/dataset_tool.py "\
  "--source /content/drive/MyDrive/skin_data/images/images_fst6 "\
  "--dest /content/drive/MyDrive/skin_data/dataset/images_fst6"

!{CMD}

Error: --dest folder must be empty


## Clean Up your Images
All images must have the same dimensions and color depth. This code can identify images that have issues.

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/skin_data/images/images_fst6'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz

## Perform Initial Training
This code performs the initial training. Set SNAP low enough to get a snapshot before Colab forces you to quit.

In [ ]:
import os
# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/skin_data/experiments"
DATA = "/content/drive/MyDrive/skin_data/dataset/images_fst6"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py "\
  f"--snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/skin_data/dataset/images_fst6",
    "use_labels": false,
    "max_size": 319,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 1

## Resume Training
You can now resume training after you are interrupted by something in the pervious step.

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/skin_data/experiments"
NETWORK = "network-snapshot-000040.pkl"
RESUME = os.path.join(EXPERIMENTS, \
                "00008-images_fst6-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/skin_data/dataset/images_fst6"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py "\
  f"--snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/skin_data/dataset/images_fst6",
    "use_labels": false,
    "max_size": 319,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 1

## Generate Images

In [25]:
# Paths and parameters
EXPERIMENTS = "/content/drive/MyDrive/skin_data/experiments"
NETWORK = "network-snapshot-000760.pkl"
RESUME = os.path.join(EXPERIMENTS, "00010-images_fst6-auto1-resumecustom", NETWORK)
OUTDIR = "/content/drive/MyDrive/skin_data/generated_images"  # Output directory for generated images

# Seeds and truncation
SEEDS = "33,65,19,253"
TRUNCATION = 0.7

# Ensure the output directory exists
os.makedirs(OUTDIR, exist_ok=True)

# Build the command for generating images
cmd = f"python /content/stylegan2-ada-pytorch/generate.py "\
      f"--network={RESUME} --outdir={OUTDIR} --seeds={SEEDS} --trunc={TRUNCATION}"

# Run the command
!{cmd}


Loading networks from "/content/drive/MyDrive/skin_data/experiments/00010-images_fst6-auto1-resumecustom/network-snapshot-000760.pkl"...
Generating image for seed 33 (0/4) ...
Setting up PyTorch plugin "bias_act_plugin"... /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Generating image for seed 65 (1/4) ...
Generating image for seed 19 (2/4) ...
Generating image for seed 253 (3/4) ...
